# LinkedIn Hiring Rate Analysis: Labor Market Dynamics (2018-2025)
The analysis of LinkedIn Hiring Rate (LHR) data reveals distinct patterns across Middle East & North Africa, Afghanistan & Pakistan (MENAAP) economies, with notable variations in labor market recovery. The countries analysed are Algeria, Bahrain, Egypt, Jordan, Morocco, Qatar, Saudi Arabia, Tunisia, United Arab Emirates

In [83]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, Tabs, TabPanel, Toggle, CustomJS
from bokeh.palettes import Category20
from bokeh.layouts import column
import openpyxl
import statsmodels.formula.api as smf

In [84]:
def clean_linkedin_data(excel_file):
    """Clean and prepare LinkedIn data from Excel file."""
    try:
        data_corrected = pd.read_excel(excel_file, sheet_name="2A - LHR by Ctry", header=3)

        data_cleaned = data_corrected.rename(columns={
            'Unnamed: 1': 'Month',
            'Unnamed: 2': 'Country',
            'Unnamed: 3': 'LHR (YOY)'
        })

        data_cleaned = data_cleaned.dropna(subset=['Month', 'Country', 'LHR (YOY)'])
        data_cleaned = data_cleaned.iloc[1:]
        data_cleaned = data_cleaned.drop(columns=['Unnamed: 0'])

        # Clean up whitespace and standardize country names
        data_cleaned["Country"] = data_cleaned["Country"].str.strip()
        data_cleaned["Country"] = data_cleaned["Country"].replace({
            "Turkey": "Turkiye",
            "Türkiye": "Turkiye"
        })

        return data_cleaned

    except Exception as e:
        print(f"Error during data cleaning: {str(e)}")
        return None



In [85]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import (
    ColumnDataSource, HoverTool, Tabs, TabPanel, Toggle, CustomJS, Button
)
from bokeh.layouts import column
from bokeh.palettes import Category20
import pandas as pd

def create_country_plots(excel_file):
    """Create interactive plots showing LinkedIn Hiring Rate by Country with 2022+ filter toggle and download button."""
    output_notebook()
    
    country_map = {
        'DZ': 'Algeria', 
        'BH': 'Bahrain', 
        'EG' : 'Egypt',
        'IQ': 'Iraq',
        'JO': 'Jordan',
        'MA': 'Morocco',
        'QA': 'Qatar',
        'SA': 'Saudi Arabia',
        'TN': 'Tunisia',
        'AE': 'United Arab Emirates',
        'PK': 'Pakistan',
        'US': 'United States'
    }
    
    try:
        df = clean_linkedin_data(excel_file)
        
        if df is None:
            print("Data cleaning failed. Please check your Excel file.")
            return
        
        # Compute global y-axis range for all countries
        global_min = df['LHR (YOY)'].min()
        global_max = df['LHR (YOY)'].max()

        tabs = []

        for idx, (country_code, country_name) in enumerate(country_map.items()):
            country_data = df[df['Country'] == country_name].copy()
            if country_data.empty:
                print(f"No data available for {country_name}.")
                continue

            # Keep datetime format for plotting
            country_data['Month'] = pd.to_datetime(country_data['Month'])
            country_data = country_data.sort_values('Month')

            # Add string version of Month for download
            country_data['Month_str'] = country_data['Month'].dt.strftime('%Y-%m')

            # Prepare sources
            full_data = country_data
            filtered_data = full_data[full_data['Month'] >= '2022-01-01']

            source = ColumnDataSource(full_data)
            source_filtered = ColumnDataSource(filtered_data)
            # Download source: string Month only
            source_download = ColumnDataSource(
                full_data[['Country', 'Month_str', 'LHR (YOY)']].rename(columns={'Month_str': 'Month'})
            )

            # Create plot
            p = figure(
                title=f"LinkedIn Hiring Rate in {country_name}",
                x_axis_type='datetime',
                width=800,
                height=500,
                background_fill_color="#f8f9fa",
                y_range=(global_min, global_max)
            )

            p.line(
                x='Month',
                y='LHR (YOY)',
                source=source,
                line_width=3,
                color=Category20[20][idx % 20]
            )

            hover = HoverTool(tooltips=[
                ("Month", "@Month{%b %Y}"),
                ("LHR (YOY)", "@{LHR (YOY)}{0.00}%")
            ], formatters={"@Month": "datetime"}, mode='vline')
            p.add_tools(hover)

            p.xaxis.axis_label = 'Month'
            p.yaxis.axis_label = 'LinkedIn Hiring Rate (YOY %)'
            p.xaxis.axis_label_text_font_size = '12pt'
            p.yaxis.axis_label_text_font_size = '12pt'
            p.xaxis.major_label_text_font_size = '10pt'
            p.yaxis.major_label_text_font_size = '10pt'
            p.title.text_font_size = '14pt'
            p.grid.grid_line_color = "gray"
            p.grid.grid_line_alpha = 0.3

            # Toggle button
            toggle = Toggle(label="Show only from 2022", button_type="success", active=False)

            callback = CustomJS(args=dict(
                toggle=toggle,
                source=source,
                full=source.data,
                filtered=source_filtered.data
            ), code="""
                source.data = toggle.active ? filtered : full;
                source.change.emit();
                toggle.label = toggle.active ? "Show full range" : "Show only from 2022";
                toggle.button_type = toggle.active ? "warning" : "success";
            """)
            toggle.js_on_change("active", callback)

            # Download button
            download_button = Button(label="Download CSV", button_type="primary")

            download_js = CustomJS(args=dict(source=source_download, name=country_name), code="""
                const data = source.data;
                const cols = ["Country", "Month", "LHR (YOY)"];
                const nrows = data[cols[0]].length;
                let csv = cols.join(",") + "\\n";
                for (let i = 0; i < nrows; i++) {
                    let row = cols.map(col => data[col][i]);
                    csv += row.join(",") + "\\n";
                }
                const blob = new Blob([csv], { type: 'text/csv;charset=utf-8;' });
                const link = document.createElement("a");
                link.href = URL.createObjectURL(blob);
                const filename = "linkedin_HR_" + name.replace(/ /g, "_") + ".csv";
                link.download = filename;
                link.click();
            """)
            download_button.js_on_click(download_js)

            # Combine all widgetsreate_country_plots
            layout = column(toggle, download_button, p)
            tab = TabPanel(child=layout, title=country_name)
            tabs.append(tab)

        if tabs:
            show(Tabs(tabs=tabs))
        else:
            print("No data available for any of the specified countries.")
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print("Please check your Excel file structure and column names.")




In [13]:
from bokeh.io import output_notebook, show
output_notebook()
file_path = '../../data/LinkedIn/LinkedIn_LHR by Industry_Feb2025.xlsx'

create_country_plots(file_path)

Loading BokehJS ...

Loading BokehJS ...

### LinkedIn Hiring Rate Insights

#### General Note

To interpret these results appropriately, it's important to consider LinkedIn’s guidance on the level of disaggregation at which the hiring rate estimates are valid:

* For **Algeria, Bahrain, Egypt, Jordan, Morocco, Qatar, Saudi Arabia, and Tunisia**, LinkedIn deems the hiring rate valid across **tech-related industries**.
* For the **United Arab Emirates**, estimates are considered valid across **a broader set of industries**.
* For **Iraq and Pakistan**, LinkedIn only considers the estimates valid when presented at the **national level**.

The following insights are presented at the **national level** and are therefore valid for all countries.


#### COVID-19 Impact (2020)

* All markets experienced a **significant contraction** in 2020.
* Uniform negative growth rates were observed across countries (\~**-0.65%**).
* **Morocco (-0.8045%)** and **Tunisia (-0.7936%)** recorded the largest declines.
* Recovery patterns began to diverge in magnitude, though they followed similar overall trends.

#### Post-COVID Recovery Surge (2021)

* A **sharp increase in April 2021** hiring rates is visible across countries, driven by **year-over-year (YoY) calculations** ([see methodology](https://economicgraph.linkedin.com/content/dam/me/economicgraph/en-us/PDF/linkedin-hiring-rate-methodology.pdf)).

  * This spike is primarily due to **base effects** from April 2020, when hiring plummeted.
  * The surge reflects a **rebound** in labor market activity, not necessarily a hiring boom.

* Peak YoY growth rates in mid-2021:

  * **Morocco:** 3.503% (April)
  * **Algeria:** 3.4545% (April)
  * **Tunisia:** 3.118% (April)
  * **Pakistan:** 2.48% (May)
  * **Iraq:** 2.1892% (April)
  * **UAE:** 2.0987% (April)
  * **Qatar:** 2.0857% (April)
  * **Jordan:** 2.0499% (April)
  * **Saudi Arabia:** 1.9015% (April)
  * **Egypt:** 1.4289% (April)
  * **Bahrain:** 1.43% (May)

* Most countries peaked in **April**, except **Pakistan** and **Bahrain**, which peaked in **May**.

* The synchronized peaks suggest a **regional “catch-up” effect** after COVID-19 disruptions.

#### Market Stabilization Patterns (2022–2024)

* **Morocco**: Continued growth through 2022 until **December**, followed by mostly negative or near-zero growth.
* **Algeria**: Stable trend around zero; **slightly negative** since **December 2023**.
* **Tunisia**: Positive growth in 2022 with a **May peak (0.91%)**; mostly negative growth from **April 2023** onward.
* **Pakistan**: Positive growth throughout 2022 until **October 2023**, then fluctuating around zero with short periods of positive/negative growth.
* **Iraq**: Mild positive growth during 2022; slight negative or near-zero growth through 2023–2024 with a **July 2024 peak (0.12%)**, then stable around zero.
* **United Arab Emirates**: Positive growth during the **first three quarters of 2022**, followed by **slightly negative** values near zero.
* **Qatar**: Dynamic trend with early 2022 growth, a **dip in December 2022 (-0.04%)**, followed by a **peak in December 2023 (+0.47%)**, and mostly negative but near-zero values afterward.
* **Jordan**: Positive growth during 2022; **persistent negative values from January 2023 to September 2024** (except **May 2023**), followed by slight improvement post-October 2024.
* **Saudi Arabia**: Fluctuating around zero since 2022; notable variations include: **May 2022:** +0.45%, **April 2023:** -0.27%, **June 2023:** -0.26%, **July 2023:** +0.36%.
* **Egypt**: Some growth in early 2022; mostly negative from late 2022 to **April 2024**. Beginning **July 2024**, stable near-zero values with slight improvements.
* **Bahrain**: Some growth in 2022, but **almost all periods since January 2023 show negative growth**.




In [134]:
# read data 
# conflict
conflict_df =  pd.read_csv("../../data/conflict/acled_mena_raw_20250718.csv")
# linkedin
file_path = '../../data/LinkedIn/LinkedIn_LHR by Industry_Feb2025.xlsx'
linkedin_df = df = clean_linkedin_data(file_path)

# Standardize column names
conflict_df.columns = conflict_df.columns.str.strip().str.lower()

# Parse and convert event_date
conflict_df['event_date'] = pd.to_datetime(conflict_df['event_date'])

# Create 'month' column
conflict_df['month'] = conflict_df['event_date'].dt.to_period('M').dt.to_timestamp()

# Standardize country name formatting
conflict_df['country'] = conflict_df['country'].str.strip()

# remove israel 

conflict_df = conflict_df[conflict_df['country'] != 'Israel']

# A. Grouped by country, month, and event type
agg_by_type = conflict_df.groupby(['country', 'month', 'event_type']).agg(
    events=('event_type', 'count'),
    fatalities=('fatalities', 'sum')
).reset_index()

# remove filter out Protests and Strategic developments 
agg_by_type_sum = agg_by_type[~agg_by_type['event_type'].isin(['Protests', 'Strategic developments'])]

# B. Grouped by country and month (total conflict per month)
agg_by_month = conflict_df.groupby(['country', 'month']).agg(
    nr_events=('event_type', 'count'),
    total_fatalities=('fatalities', 'sum')
).reset_index()

agg_by_month_filtered = agg_by_type_sum.groupby(['country', 'month']).agg(
    nr_events=('events', 'sum'),
    total_fatalities=('fatalities', 'sum')
).reset_index()

# Standardize column names
linkedin_df.columns = linkedin_df.columns.str.strip().str.lower()

# Parse and create 'month' column
linkedin_df['month'] = pd.to_datetime(linkedin_df['month']).dt.to_period('M').dt.to_timestamp()

# Standardize country name formatting
linkedin_df['country'] = linkedin_df['country'].str.strip()

# Merge
merged_df = pd.merge(
    linkedin_df[['month', 'country', 'lhr (yoy)']],
    agg_by_month,
    on=['month', 'country'],
    how='left'  # Keep all LinkedIn data, add conflict data when available
)

merged_df_common_total = merged_df.dropna(subset=['nr_events', 'total_fatalities'])

print(merged_df_common_total.head())
print(f"Filtered dataset shape: {merged_df_common_total.shape}")

# Merge filtered data

merged_df_filtered = pd.merge(
    linkedin_df[['month', 'country', 'lhr (yoy)']],
    agg_by_month_filtered,
    on=['month', 'country'],
    how='left'  # Keep all LinkedIn data, add conflict data when available
)

merged_df_common_total_filtered = merged_df_filtered.dropna(subset=['nr_events', 'total_fatalities'])
print(merged_df_common_total_filtered.head())
print(f"Filtered dataset shape: {merged_df_common_total_filtered.shape}")




          month  country lhr (yoy)  nr_events  total_fatalities
5200 2022-06-01  Algeria    0.2522       18.0               0.0
5204 2022-06-01  Bahrain    0.3407       76.0               0.0
5219 2022-06-01    Egypt    0.2083       20.0              23.0
5232 2022-06-01     Iraq    0.3229      594.0             110.0
5237 2022-06-01   Jordan    0.4201       32.0               0.0
Filtered dataset shape: (317, 5)
          month  country lhr (yoy)  nr_events  total_fatalities
5219 2022-06-01    Egypt    0.2083       20.0              23.0
5232 2022-06-01     Iraq    0.3229      494.0             110.0
5237 2022-06-01   Jordan    0.4201        2.0               0.0
5243 2022-06-01    Malta    0.1138        1.0               0.0
5246 2022-06-01  Morocco    0.1726        7.0              25.0
Filtered dataset shape: (249, 5)


### Exploring relationship between LHR and conflict

In [114]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, Panel, Tabs
from bokeh.layouts import column
import numpy as np
from scipy.stats import linregress

output_notebook()

# Filter to positive fatalities only
merged_df_common = merged_df_common_total[merged_df_common_total['total_fatalities'] > 0]

# Prepare Bokeh tabs
tabs = []

for country in merged_df_common['country'].unique():
    df_country = merged_df_common[merged_df_common['country'] == country]

    # Require at least 3 data points to compute correlation and regression
    if df_country.shape[0] < 3:
        continue

    # Compute Pearson correlation
    corr = df_country[['total_fatalities', 'lhr (yoy)']].corr().iloc[0, 1]
    corr_text = f"Pearson r = {corr:.2f}"

    # Regression coefficients: y = m*x + b
    x = df_country['total_fatalities']
    y = pd.to_numeric(df_country['lhr (yoy)'], errors='coerce')
    m, b = np.polyfit(x, y, 1)

    # Generate line values
    x_fit = np.linspace(x.min(), x.max(), 100)
    y_fit = m * x_fit + b

    # Bokeh data sources
    source = ColumnDataSource(df_country)
    reg_source = ColumnDataSource(data={'x': x_fit, 'y': y_fit})
    
    regression_eq = f"y = {m:.4f}x + {b:.4f}"

    # Create figure
    p = figure(
        title=f"{country} – Conflict Fatalities vs Hiring Rate\n{corr_text}",
        x_axis_label="Total Fatalities",
        y_axis_label="LinkedIn Hiring Rate (YOY)",
        width=500,
        height=400,
        tools="pan,wheel_zoom,box_zoom,reset,hover,save"
    )

    # Scatter
    p.scatter(x='total_fatalities', y='lhr (yoy)', source=source, size=8, alpha=0.6)

    # Regression line
    p.line(
    x='x', y='y',
    source=reg_source,
    line_width=2,
    color='firebrick',
    legend_label=regression_eq
)

    # Hover tool
    p.hover.tooltips = [
        ("Month", "@month"),
        ("Fatalities", "@total_fatalities"),
        ("LHR (YOY)", "@{lhr (yoy)}")
    ]

    p.legend.visible = True

    tab = TabPanel(child=column(p), title=country)
    tabs.append(tab)

# Show tabs
show(Tabs(tabs=tabs))


Loading BokehJS ...


#### Correlation Between Conflict Fatalities and LinkedIn Hiring Rate

We explored the relationship between conflict-related fatalities and the LinkedIn Hiring Rate (YOY) across several countries using monthly data.

Overall, **we do not observe a strong or consistent negative correlation** — that is, **higher fatalities do not clearly correspond to lower hiring growth**.

**Country-Level Pearson Correlation Coefficients**

| Country          | Pearson r | Interpretation                 |
| ---------------- | --------- | ------------------------------ |
| **Egypt**        | +0.24     | Weak positive correlation      |
| **Iraq**         | +0.23     | Weak positive correlation      |
| **Morocco**      | +0.18     | Very weak positive correlation |
| **Saudi Arabia** | +0.19     | Very weak positive correlation |
| **Tunisia**      | +0.08     | Very weak positive correlation |
| **Pakistan**     | -0.12     | Very weak negative correlation |
| **Jordan**       | -0.17     | Very weak negative correlation |
| **Algeria**      | -0.23     | Weak negative correlation      |

> None of the countries show a strong negative correlation. In fact, most exhibit weak or even positive correlations, suggesting that **conflict severity (as measured by fatalities) may not be a reliable predictor of changes in hiring rate** based on LinkedIn data.


In [131]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, Panel, Tabs
from bokeh.layouts import column
import numpy as np
from scipy.stats import linregress

output_notebook()

# Filter to positive events only
merged_df_common = merged_df_common_total[merged_df_common_total['nr_events'] > 0]

# Prepare Bokeh tabs
tabs = []

for country in merged_df_common['country'].unique():
    df_country = merged_df_common[merged_df_common['country'] == country]

    # Require at least 3 data points to compute correlation and regression
    if df_country.shape[0] < 3:
        continue

    # Compute Pearson correlation
    corr = df_country[['nr_events', 'lhr (yoy)']].corr().iloc[0, 1]
    corr_text = f"Pearson r = {corr:.2f}"

    # Regression coefficients: y = m*x + b
    x = df_country['nr_events']
    y = pd.to_numeric(df_country['lhr (yoy)'], errors='coerce')
    m, b = np.polyfit(x, y, 1)

    # Generate line values
    x_fit = np.linspace(x.min(), x.max(), 100)
    y_fit = m * x_fit + b

    # Bokeh data sources
    source = ColumnDataSource(df_country)
    reg_source = ColumnDataSource(data={'x': x_fit, 'y': y_fit})
    
    regression_eq = f"y = {m:.4f}x + {b:.4f}"

    # Create figure
    p = figure(
        title=f"{country} – Conflict Events vs Hiring Rate\n{corr_text}",
        x_axis_label="Total Events",
        y_axis_label="LinkedIn Hiring Rate (YOY)",
        width=500,
        height=400,
        tools="pan,wheel_zoom,box_zoom,reset,hover,save"
    )

    # Scatter
    p.scatter(x='nr_events', y='lhr (yoy)', source=source, size=8, alpha=0.6)

    # Regression line
    p.line(
    x='x', y='y',
    source=reg_source,
    line_width=2,
    color='firebrick',
    legend_label=regression_eq
)

    # Hover tool
    p.hover.tooltips = [
        ("Month", "@month"),
        ("Events", "@nr_events"),
        ("LHR (YOY)", "@{lhr (yoy)}")
    ]

    p.legend.visible = True

    tab = TabPanel(child=column(p), title=country)
    tabs.append(tab)

# Show tabs
show(Tabs(tabs=tabs))

Loading BokehJS ...

#### Correlation Between Conflict Number of Events and LinkedIn Hiring Rate

We examined the correlation between the **number of conflict events** and the **LinkedIn Hiring Rate (YOY)** across countries. Overall, **there is no consistent pattern** — most countries show weak or no correlation.

**Country-Level Pearson Correlation Coefficients**

| Country                  | Pearson r | Interpretation                                        |
| ------------------------ | --------- | ----------------------------------------------------- |
| **Algeria**              | +0.17     | Very weak positive correlation                        |
| **Bahrain**              | -0.05     | Very weak negative correlation                        |
| **Egypt**                | -0.08     | Very weak negative correlation                        |
| **Iraq**                 | +0.27     | Weak positive correlation                             |
| **Jordan**               | -0.12     | Very weak negative correlation                        |
| **Malta**                | +0.16     | Very weak positive correlation                        |
| **Morocco**              | -0.54     | Moderate negative correlation                         |
| **Pakistan**             | +0.07     | Very weak positive correlation                        |
| **Saudi Arabia**         | +0.32     | Weak positive correlation                             |
| **Tunisia**              | +0.03     | Very weak positive correlation                        |
| **Qatar**                | -0.44     | Moderate negative correlation                         |
| **United Arab Emirates** | +0.91     | Very strong positive correlation (⚠️ few data points) |

> With the **exception of Morocco, Qatar, and the UAE**, most countries show only weak or inconsistent relationships.
> The UAE's very high correlation should be interpreted cautiously due to **a small number of observations**.

The same pattern was observed excluding event types "Protests" and "Strategic developments"




#### Regression analysis

In [142]:
from statsmodels.iolib.summary2 import summary_col

# Ensure lhr (yoy) is numeric
merged_df_common_total = merged_df_common_total.copy()
merged_df_common_total['lhr (yoy)'] = pd.to_numeric(merged_df_common_total['lhr (yoy)'], errors='coerce')

# Run models
model_1 = smf.ols("Q('lhr (yoy)') ~ total_fatalities", data=merged_df_common_total).fit()
model_2 = smf.ols("Q('lhr (yoy)') ~ nr_events", data=merged_df_common_total).fit()
model_3 = smf.ols("Q('lhr (yoy)') ~ total_fatalities + nr_events", data=merged_df_common_total).fit()
model_4 = smf.ols("Q('lhr (yoy)') ~ total_fatalities + nr_events + C(country) + C(month)", data=merged_df_common_total).fit()

# Define whether fixed effects are included
def has_country_fe(model):
    return "Yes" if any("C(country)" in var for var in model.params.index) else "No"

def has_month_fe(model):
    return "Yes" if any("C(month)" in var for var in model.params.index) else "No"

# Create the formatted summary table
regression_table = summary_col(
    results=[model_1, model_2, model_3, model_4],
    float_format="%.4f",
    stars=True,
    model_names=[
        "Model 1: Fatalities Only",
        "Model 2: Events Only",
        "Model 3: Fatalities + Events",
        "Model 4: + Fixed Effects"
    ],
    info_dict={
        'N': lambda x: f"{int(x.nobs)}",
        'R-squared': lambda x: f"{x.rsquared:.3f}",
        'Country FE': has_country_fe,
        'Month FE': has_month_fe
    }
)

# Print table without fixed effect dummies
print(regression_table)







                                             Model 1: Fatalities Only Model 2: Events Only Model 3: Fatalities + Events Model 4: + Fixed Effects
------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                    -0.0318***               -0.0324***           -0.0315***                   0.3560***               
                                             (0.0101)                 (0.0109)             (0.0110)                     (0.0360)                
total_fatalities                             0.0001                                        0.0001                       0.0000                  
                                             (0.0001)                                      (0.0002)                     (0.0002)                
nr_events                                                             0.0000               -0.0000                      0.0001   

## Summary of Regression Results

We estimate the relationship between conflict intensity and LinkedIn Hiring Rate (LHR) across countries and months using four OLS models:

* **Model 1** includes only total fatalities.
* **Model 2** includes only number of events.
* **Model 3** includes both.
* **Model 4** adds country and month fixed effects.

**Key takeaways:**

* Models 1–3 show **very low explanatory power** (R² < 0.01), suggesting that fatalities or number of events alone **do not meaningfully explain hiring rate changes**.
* **Model 4**, which includes **fixed effects**, improves the model fit substantially (R² = 0.627), highlighting the importance of controlling for **country-specific and time-specific factors**.
* The coefficient for **number of events** becomes statistically significant in Model 4, but the effect size remains **very small**.
* **Total fatalities are not statistically significant** in any specification.

Overall, conflict indicators appear to have **limited explanatory power** for variations in LHR when not controlling for fixed effects. Most of the variation is captured by **country and month effects**, not by conflict itself.